# Impelemntation of Recency Aware CF Filtering for the Next Basket Recommendation

## 0-Modules & Imports

For a start we going to install and imports some important python modules we going to work with:
* We first, using the package installer Pip, we install similaripy module. 
* **similaripy** : is a fast Python implementation for similarity algorithms using sparse matrices useful in CF-Recommender Systems

In [ ]:
!pip install similaripy

* Once the installation is done, we imports the needed python modules


In [ ]:
# Imports
import os
import time
import numpy as np
import pandas as pd
# In order to deal with large sparse matrix we need to compresse them using
# the sparse sub_module of scipy lib
from scipy import sparse
from sklearn.metrics import ndcg_score
import similaripy as sim

## If Working with Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

And navigate where data is stored 

In [ ]:
%cd /content/drive/"My Drive"/"For colab"/RecomdSys

## I- Data Manipulation:
In this, first section, I started by manipulating the data we've been given, Instacart & Dunnhumby datasets, by creating two parsers :
* instacartParser
* DunnhumbyParser (Not yet)

In parsers, I've done some light preprocessing:
  
* Removing all items that appear in less than some number of baskets (aka item_threshold).
* Excluding users with less than some number of baskets (aka user_threshold)

And in order to deal with limitation of my computer setting (Ram 16GB/i7-4501U CPU), I also added a subdata percentage parametre to extract a certain amount of data to deal with.  

In [ ]:
def instacartParser(dataPath, item_threshold=10, basket_threshold=2, subdata=0.1,verbose=True):
  '''
  IN:
    dataPath : os path to instacart data
    item_threshold : (default = 10 basket)
    basket_threshold : (default = 2 basket)
    subdata : (default: 10% of data)
    verbose: Boolean, (default:True)
  OUT:
    df_train : DataFrame where columns = [BID, UID, order,articles]
    dev_set, test_set : user-baskets items dict like {UID -> [PID,...], UID -> [PID,...], ...}
    df_products: DataFrame where columns = [PID, description, department, category]
  '''
  if verbose :
    start = time.time()
    # Start Time for calculating execution time
    print('Constructing products DataFrame object ...')

  # Read products.csv
  df_products= pd.read_csv(os.path.join(path,"products.csv"))
  df_products.columns = ['PID', 'description', 'categoryId', 'departmentId']
  # Read departments.csv and merge
  tmp = pd.read_csv(os.path.join(path,"departments.csv"))
  tmp.columns = ['departmentId', 'department']
  df_products = pd.merge(df_products, tmp, on='departmentId')
  # Read aisles.csv and merge 
  tmp = pd.read_csv(os.path.join(path,'aisles.csv'))
  tmp.columns = ['categoryId', 'category']
  df_products = pd.merge(df_products, tmp, on='categoryId')[['PID', 'description','department','category']]
  del tmp

  # preprocessing
  if verbose:
    print('Constructing order_products DataFrame object ...')

  df_order_products_prior = pd.read_csv(os.path.join(path,"order_products__prior.csv"))
  df_order_products_train = pd.read_csv(os.path.join(path,"order_products__train.csv"))
  df_order_products = pd.concat([df_order_products_prior, df_order_products_train])[['order_id', 'product_id']]
  df_order_products.columns= ['BID','PID']
  del df_order_products_prior, df_order_products_train,

  if verbose:
    print('Filtring items ...')
  # Remouving all items that appears in less than item_threshold baskets
  products_count = df_order_products['PID'].value_counts()
  df_order_products= df_order_products.loc[df_order_products['PID'].isin(products_count[products_count >= item_threshold].index)]
  del products_count
  # Updating production list 
  pd.merge(df_products,df_order_products['PID'],on='PID')

  if verbose:
    print('Reading Users order.csv file ...')
  df_orders = pd.read_csv(os.path.join(path,"orders.csv"))[['order_id', 'user_id', 'order_number', 'eval_set']]
  df_orders.columns = ['BID','UID','order', 'set']

  if verbose:
    print('Filtring Users...')
    print('Getting',subdata*100,'% of our dataset...')
  # User filtring
  # Remouving users with less than basket_threshold baskets
  user_count = df_orders['UID'].value_counts()
  user_filter = user_count[(user_count>=basket_threshold)&(np.random.rand(len(user_count))< subdata)]
  del user_count
  df_orders = df_orders[df_orders['UID'].isin(user_filter.index)]
  del user_filter

  # Reset UID 
  if verbose:
    print('Reset UID indexing')

  user_dict = dict(zip(df_orders['UID'].unique(),range(len(df_orders['UID'].unique()))))
  df_orders['UID'] = df_orders['UID'].map(user_dict)
  del user_dict
  # reset product index
  df_order_products = df_order_products.loc[df_order_products['BID'].isin(df_orders['BID'].unique())]
  df_products = df_products[df_products['PID'].isin(df_order_products['PID'].unique())]  
  product_dict = dict(zip(df_order_products['PID'].unique(),range(len(df_order_products['PID'].unique()))))
  df_products['PID'] = df_products['PID'].map(product_dict)
  df_order_products['PID'] = df_order_products['PID'].map(product_dict)
  del product_dict

  # Join Tables
  if verbose:
    print('Joining tables ...')
  df_data = pd.merge(df_orders, df_order_products, on= 'BID')
  del df_orders, df_order_products

  if verbose:
    print("spliting data ...")
  # Setting last baskets as dev/test sets
  last_basket_indexes = df_data.iloc[df_data.groupby(['UID'])['order'].idxmax()]['BID'].values
  df_data.loc[df_data['BID'].isin(last_basket_indexes),'set']='test'
  df_data.loc[df_data['set']=='prior', 'set'] = 'train'
  del last_basket_indexes

  # train test split data
  df_split = df_data[df_data['set']=='test'].groupby(by=['UID'])['PID'].apply(list).reset_index(name='articles')
  msk = (np.random.rand(len(df_split))<0.5)
  df_dev, df_test = df_split[msk], df_split[~msk]
  del df_split

  df_train = df_data[df_data['set']=='train'][['UID','BID','order','PID']]
  dev_set = dict(zip(df_dev['UID'],df_dev['articles']))
  test_set = dict(zip(df_test['UID'],df_test['articles']))

  del msk, df_dev, df_test
  # simple check
  assert (len(dev_set)+len(test_set))==(df_data['UID'].unique().shape[0])
  del df_data

  if verbose:
    print("processing took {0:.1f} sec".format(time.time() - start))
  
  return df_train, dev_set, test_set, df_products

## II- Helper functions:

* Top-k Predection 
* Evaluation using the nDGC@k score provided by sklearn.metrics python module

In [ ]:
def top_n(row, n):
    '''
    IN : 
      row : 1-D ndarray
      n   : int, number of top items
    OUT:
      top_values  : 1-D ndarray, Represent Top-n scores of the given row
      top_indices : 1-D ndarray, Represent Top-n users indices of the given row
    '''
    # Get user indices to sort the given row
    top_indices = row.argsort()[-n:][::-1]
    # Use the top_indices to get top_values score
    top_values  = row[top_indices]
    return top_values, top_indices

def prediction(predMat, k):
    '''
    In :
      predMat : the predection matrix 
      {UWPop, UB-CF, IB-CF }with/out recency (@r)
    Out :
      score, pred : ndarray of shape =(n_users, k)
      retun the top-k score and predection matrix
    '''
    n_users = predMat.shape[0]
    score = np.zeros((n_users, k))
    pred  = np.zeros((n_users, k))
    for i in range(n_users):
        score[i], pred[i] = top_n(predMat[i],k)
    return score.astype('float64'), pred.astype('int64')

def evaluation(score, pred, test_set, dev_set, k=5):
    '''
    Calculate the ndgs score for both test/dev sets for a giver user-item score and predection matrix.
    IN:
      score, pred: (n_user, m_items) ndarray matrix
      test_set, dev_set : user-baskets items dict like {UID -> [PID,...], UID -> [PID,...], ...}
      k :(default fixed to 5)    
    OUT:
      test_ndcg_score, dev_ndcg_score : type:int, evaluation metric for both test and dev set respectively  
    '''
    # Get the test and dev set User IDs
    test_keys = test_set.keys()
    dev_keys  = dev_set.keys()
    # Construct the True_relecvance and score vectors
    true_relevance_test = np.asarray([np.isin(pred[key],test_set[key]).astype(int) for key in test_keys])
    true_relevance_dev  = np.asarray([np.isin(pred[key],dev_set[key]).astype(int) for key in dev_keys])
    score_test = score[list(test_keys)]
    score_dev  = score[list(dev_keys)]
    # Calculate the ndgc@k evaluation metric 
    test_ndcg_score = ndcg_score(true_relevance_test, score_test, k)
    dev_ndcg_score  = ndcg_score(true_relevance_dev, score_dev, k)
    return test_ndcg_score, dev_ndcg_score

## III- Methods implementation

Begin by cleaning and splitting Our Data

In [ ]:
# here we hard-coded the instacart dataset's path
path = "instacart"
# We return df_products for further uses like doing some analysis 
df_train, dev_set, test_set, df_products = instacartParser(path, item_threshold=10, basket_threshold=2, subdata=0.05,verbose=True)

Show some dimensions

In [ ]:
n_users = df_train.UID.unique().shape[0]
n_items = df_products['PID'].unique().shape[0]
n_baskets = df_train.BID.unique().shape[0]
print('n_users:',n_users)
print('n_items:',n_items)
print('n_baskets:',n_baskets)

### 1-User-Wise Popularity Method

In [ ]:
def uwPopMat(df_train, n_items, recency=0):
  '''
    Calculate the user popularity matrix with the given recency window
  '''
  n_users = df_train.UID.unique().shape[0]
  if (recency>0):
    # Get the number of user baskets Bu 
    BUCount = df_train.groupby(['UID'])['order'].max().reset_index(name='Bu')
    # Calculate the denominator which equal to Min(recency,Bu) for each user
    BUCount['denominator'] = np.minimum(BUCount['Bu'],5)
    # Calculater the order index, form where we start counting item appearance in recent orders   
    BUCount['startindex'] = np.maximum(BUCount['Bu']-5,0)
    # Calcualte item appearance in recent orders   
    tmp = pd.merge(BUCount, df_train,on='UID')[['UID','PID','order','startindex']]
    tmp = tmp.loc[(tmp['order']>=tmp['startindex'])==True].groupby(['UID','PID'])['order'].count().reset_index(name='numerator')
    tmp = pd.merge(BUCount[['UID','denominator']],tmp,on='UID')
    # finally calculate the recency aware user-wise popularity
    tmp['Score'] = tmp['numerator']/tmp['denominator']
  else : 
    # Calculate user-wise popularity for each item
    BUCount = df_train.groupby(['UID'])['order'].max().reset_index(name='Bu')
    BUICount = df_train.groupby(['UID','PID'])['BID'].count().reset_index(name='Bui')
    tmp = pd.merge(BUICount, BUCount, on='UID')
    del BUICount
    tmp['Score'] = tmp['Bui']/tmp['Bu']
    del BUCount
    # get the 3 columns needed to construct our user-wise Popularity matrix
  df_UWpop =  tmp[['UID','PID','Score']]   
  del tmp
  # Generate user-wise popularity matrix in COOrdinate format
  UWP_mat = sparse.coo_matrix((df_UWpop.Score.values, (df_UWpop.UID.values, df_UWpop.PID.values)), shape=(n_users,n_items))
  del df_UWpop
  return sparse.csr_matrix(UWP_mat)

#### A - UWPop Without recency

##### Code:

In [ ]:
n_items = df_products['PID'].unique().shape[0]
UWP_mat = uwPopMat(df_train, n_items, recency=0)

In [ ]:
UWP_mat.shape

##### Evaluation

In [ ]:
# prediction 
score, pred = prediction(UWP_mat.toarray(), k=10)
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=10)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

#### b-UWPop with recency (UWPop@r)

##### Code:

In [ ]:
n_items = df_products['PID'].unique().shape[0]
UWPr_mat = uwPopMat(df_train, n_items, recency=10)

In [ ]:
UWPr_mat.shape

##### Evaluation

In [ ]:
# predection
score, pred = prediction(UWPr_mat.toarray(), k=10)
# Evaluation
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=10)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

### sparse martrix
Due to the high memory consumption and in order to build the User-user (item-item) similarity matrix, which is known to be highly sparse. we always return a sparse UWPop (user-wise popularity) matrix, using the scipy-python sub-module **sparse** and **similaripy** python module.

First we start by getting the compressed UWPop Matrix (with and without recency) in CSR(Compressed Sparse Row) format. Then calculate the similarity matrix in order to get the final prediction and evaluate our model. 

 

### 2-Popularity-based Collaborative Filtering

#### 2.1- User Popularity-based CF (UP-CF)

In [ ]:
def upcf(df_train, UWP_sparse, n_items, alpha = 0.25 ,q=5, k=10):
  n_users = df_train['UID'].unique().shape[0]
  df_user_item = df_train.groupby(['UID','PID']).size().reset_index(name="bool")[['UID','PID']]
  # Generate the User_item matrix using the parse matrix COOrdinate format.
  userItem_mat = sparse.coo_matrix((np.ones((df_user_item.shape[0])), (df_user_item.UID.values, df_user_item.PID.values)), shape=(n_users,n_items))
  # Calculate the asymmetric similarity cosine matrix 
  userSim = sim.asymmetric_cosine(sparse.csr_matrix(userItem_mat), alpha=0.25, k=10)
  # recommend k items to users
  user_recommendations = sim.dot_product(userSim.power(5), UWP_sparse, k=10)
  return user_recommendations

##### A- UP-CF without recency

In [ ]:
n_items = df_products['PID'].unique().shape[0]
user_recommendations = upcf(df_train, UWP_mat, n_items, alpha = 0.25, q=5, k=10)

In [ ]:
# predection
score, pred = prediction(user_recommendations.toarray(), k=10)
del user_recommendations
# Evaluation
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=10)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

##### B- UP-CF with recency (UP-CF@r)

In [ ]:
n_items = df_products['PID'].unique().shape[0]
user_recommendations = upcf(df_train, UWPr_mat, n_items, alpha = 0.25, q=5, k=10)

In [ ]:
user_recommendations.shape

In [ ]:
# predection
score, pred = prediction(user_recommendations.toarray(), k=5)
del user_recommendations
# Evaluation
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=5)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

####2.2-Item popularity-based Collaborative Filtring 

In [ ]:
def ipcf(df_train, UWP_sparse, n_items,alpha = 0.25, q=5, k=10):
  # Construct the item-basket sparse matrix 
  idMax_basket = df_train.BID.max()+1
  item_basket_mat = sparse.coo_matrix((np.ones((df_train.shape[0]),dtype=int), (df_train.PID.values, df_train.BID.values)), shape=(n_items,idMax_basket))
  # Convert it to Compressed Sparse Row format to exploit its efficiency in arithmetic operations 
  sparse_mat = sparse.csr_matrix(item_basket_mat)
  # Caculate the Asymetric Cosine Similarity matrix
  itemSimMat = sim.asymmetric_cosine(sparse_mat, None, alpha, k)
  # recommend k items to users
  UWP_sparse.shape, itemSimMat.shape
  user_recommendations = sim.dot_product(UWP_sparse, itemSimMat.power(q), k)
  return user_recommendations

##### A- IP-CF without recency

In [ ]:
user_recommendations = ipcf(df_train, UWP_mat, n_items, alpha = 0.25, q=5, k=5)

In [ ]:
# predection
score, pred = prediction(user_recommendations.toarray(), k=5)
del user_recommendations
# Evaluation
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=5)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

##### B- IP-CF with recency (IP-CF@r) 

In [ ]:
# Use the UWP@r sparse matrix
user_recommendations = ipcf(df_train, UWPr_mat, n_items, alpha = 0.25, q=5, k=5)

In [ ]:
# predection
score, pred = prediction(user_recommendations.toarray(), k=5)
del user_recommendations
# Evaluation
test_ndcg, dev_ndcg = evaluation(score, pred, test_set, dev_set, k=5)
print("test score:",test_ndcg,"\n dev score:",dev_ndcg)
del score,pred

## IV- References:

**scipy sparse matrix**

ref\[0\] : https://en.wikipedia.org/wiki/Sparse_matrix

ref\[1\] : https://stackoverflow.com/questions/36135927/get-top-n-items-of-every-row-in-a-scipy-sparse-matrix

ref\[2\] : https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices

ref\[4\] : https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html

**similaripy**

ref\[5\] : https://pypi.org/project/similaripy/